In [ ]:
# Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_selection import RFE
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix, accuracy_score, balanced_accuracy_score
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import KFold, RepeatedStratifiedKFold, StratifiedKFold
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
import lightgbm as lgb
import catboost
import shap
from catboost import CatBoostClassifier


In [ ]:
def evaluate_model_test(model, X_test, y_test):
    from sklearn import metrics

    # Predict Test Data 
    y_pred = model.predict_proba(X_test)[:,1]
    for i in range(len(y_pred)):
        if y_pred[i]>0.5:
            y_pred[i]=1
        else:
            y_pred[i]=0
    

    # Calculate accuracy, precision, recall, f1-score, and kappa score
    acc = metrics.accuracy_score(y_test, y_pred)
    prec = metrics.precision_score(y_test, y_pred)
    rec = metrics.recall_score(y_test, y_pred)
    f1 = metrics.f1_score(y_test, y_pred)

    # Calculate area under curve (AUC)
    y_pred_proba = model.predict_proba(X_test)[::,1]
    fpr, tpr, _ = metrics.roc_curve(y_test, y_pred_proba)
    auc = metrics.roc_auc_score(y_test, y_pred_proba)
    
    #MCC
    mcc=matthews_corrcoef(y_test, y_pred)
    
    # Display confussion matrix
    cm = metrics.confusion_matrix(y_test, y_pred)
    total=sum(sum(cm))
    
    #accuracy=(cm[0,0]+cm[1,1])/total
    spec = cm[0,0]/(cm[0,1]+cm[0,0])
    sen= cm[1,1]/(cm[1,0]+cm[1,1])

    return {'acc': acc, 'prec': prec, 'rec': rec, 'f1': f1, 'mcc':mcc,
            'fpr': fpr, 'tpr': tpr, 'auc': auc, 'cm': cm, 'sen': sen, 'spec':spec}

In [ ]:
def evaluate_model_train(model, X_train, y_train):
    from sklearn import metrics
    conf_matrix_list_of_arrays = []
    mcc_array=[]
    #cv = KFold(n_splits=5)
    #cv = StratifiedKFold(n_splits=5)
    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1)
    lst_accu = []
    AUC_list=[]
    prec_train=np.mean(cross_val_score(model, X_train, y_train, cv=cv, scoring='precision'))
    recall_train=np.mean(cross_val_score(model, X_train, y_train, cv=cv, scoring='recall'))
    f1_train=np.mean(cross_val_score(model, X_train, y_train, cv=cv, scoring='f1'))
    Acc=np.mean(cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy'))
    print(Acc)
    for train_index, test_index in cv.split(X_train, y_train): 
        X_train_fold, X_test_fold = X_train[train_index], X_train[test_index] 
        y_train_fold, y_test_fold = y_train[train_index], y_train[test_index] 
        model.fit(X_train_fold, y_train_fold) 
        lst_accu.append(model.score(X_test_fold, y_test_fold))
        acc=np.mean(lst_accu)
        
        conf_matrix = confusion_matrix(y_test_fold, model.predict(X_test_fold))
        conf_matrix_list_of_arrays.append(conf_matrix)
        cm = np.mean(conf_matrix_list_of_arrays, axis=0)
        mcc_array.append(matthews_corrcoef(y_test_fold, model.predict(X_test_fold)))
        mcc=np.mean(mcc_array, axis=0)
        
        AUC=metrics.roc_auc_score( y_test_fold, model.predict_proba(X_test_fold)[:,1])
        AUC_list.append(AUC)
        auc=np.mean(AUC_list)
        
        
    total=sum(sum(cm))
    accuracy=(cm[0,0]+cm[1,1])/total
    specificity = cm[0,0]/(cm[0,1]+cm[0,0])
    sensitivity = cm[1,1]/(cm[1,0]+cm[1,1])
       
    
    return {'prec_train': prec_train, 'recall_train': recall_train, 'f1_train': f1_train, 'cm': cm, 'mcc': mcc,'Acc':Acc,
           'sen':sensitivity,'spec':specificity, 'acc':acc, 'lst_accu':lst_accu, 'AUC':auc}

In [ ]:
# Read feature descriptors of training data
df1 = pd.read_csv(" /path......")
df2 = pd.read_csv(" /path......")
.
.
.
df6

In [ ]:
data1=pd.concat([df1, ....],axis = 1)
X_train = data1.iloc[:,1:].values

In [ ]:
X_train=np.array(X_train)
lab=len(X_train)/2
pos_labels = np.ones(int(lab))
neg_labels = np.zeros(int(lab))
y_train = np.concatenate((pos_labels,neg_labels),axis=0)

In [ ]:
data_only = X_train
labels = y_train

In [ ]:
# Read feature descriptors of independent data
df1 = pd.read_csv(" /path......")
df2 = pd.read_csv(" /path......")
.
.
.
df6

In [ ]:
# Concatenate Independent data
data2=pd.concat([df1, ....],axis = 1)
X_test_ind = data2.iloc[:,1:].values

In [ ]:

X_test_ind=np.array(X_test_ind)
lab=len(X_test_ind)/2
pos_labels = np.ones(int(lab))
neg_labels = np.zeros(int(lab))
y_test_ind = np.concatenate((pos_labels,neg_labels),axis=0)

# Random Forest

In [ ]:
import optuna
from sklearn.ensemble import RandomForestClassifier
def RF_objective(trial):
    
    
    folds=10
    scores = []
    test_accs = []
   
    kf = KFold(n_splits=folds, shuffle=True, random_state=72)
    
    # kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=92)
    for i, (train_index, test_index) in enumerate(kf.split(data_only,labels)):
        
        
    
        X_train0, X_test1 = data_only[train_index], data_only[test_index]
        y_train0, y_test1 = labels[train_index], labels[test_index]
    
        X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0, y_train0, test_size=0.01, random_state=72, shuffle=True)
        
       
    
    n_estimators = trial.suggest_int("n_estimators", 2, 1000)
    criterion =  trial.suggest_categorical('criterion', ['entropy','gini' ])
    max_depth = trial.suggest_int('max_depth', 2, 80)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 2, 40)
    min_samples_split= trial.suggest_int("min_samples_split", 2, 40)
    min_samples_leaf= trial.suggest_int('min_samples_leaf', 2, 40)
    max_features= trial.suggest_categorical('max_features', ['auto','sqrt','log2'])
   
    ## Create Model
    rf_model = RandomForestClassifier(max_depth = max_depth, min_samples_split=min_samples_split, 
                                      min_samples_leaf = min_samples_leaf, n_estimators = n_estimators,
                                   max_features=max_features, random_state=72)


    rf_model.fit(X_train1,  y_train1)    
    y_pred = rf_model.predict(X_test1)
    accuracy = accuracy_score(y_test1, y_pred)
    return accuracy

#Execute optuna and set hyperparameters
RF_study = optuna.create_study(direction='maximize')
RF_study.optimize(RF_objective, n_trials=1000)


In [ ]:
print(RF_study.best_trial)  # Show the best value.

In [ ]:
optimized_RF=RandomForestClassifier(**RF_study.best_params)

In [ ]:
# Evaluate Model on Training data
train_eval = evaluate_model_train(optimized_RF, X_train, y_train)
print("Confusion Matrix is: ", train_eval['cm'])
print ('Accuracy : ', train_eval['Acc'])
print('Sensitivity : ', train_eval['sen'])
print('Specificity : ', train_eval['spec'])
print("Mean of Matthews Correlation Coefficient is: ", train_eval['mcc'])
print("The Acc value from CM is: ", train_eval['acc'])
print("The Recall value is: ", train_eval['recall_train'])
print("The F1 score is: ", train_eval['f1_train'])
print('The area under curve is:', train_eval['AUC'])
#print('5 accuracies: ', train_eval['lst_accu'])

In [ ]:
# Evaluate Model on Testing data
#rfc.fit(X_train, y_train)
dtc_eval = evaluate_model_test(optimized_RF, X_test, y_test)
# Print result
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('F1 Score:', dtc_eval['f1'])
print('Area Under Curve:', dtc_eval['auc'])
print('Sensitivity : ', dtc_eval['sen'])
print('Specificity : ', dtc_eval['spec'])
print('MCC Score : ', dtc_eval['mcc'])
print('Confusion Matrix:\n', dtc_eval['cm'])

# ExtraTreeClassifier

In [ ]:
def objective(trial):
    
    
    folds=10
    scores = []
    test_accs = []
   
    kf = KFold(n_splits=folds, shuffle=True, random_state=72)
    
    # kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=92)
    for i, (train_index, test_index) in enumerate(kf.split(data_only,labels)):
        X_train0, X_test1 = data_only[train_index], data_only[test_index]
        y_train0, y_test1 = labels[train_index], labels[test_index]
    
        X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0, y_train0, test_size=0.001, random_state=72, shuffle=True)
        
       
    ### define params grid to search maximum accuracy
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, step=5)
    max_depth = trial.suggest_int('max_depth', 5, 80, step=1)
    max_leaf_nodes = trial.suggest_int('max_leaf_nodes', 6, 30)
    criterion = trial.suggest_categorical('criterion', ['log_loss','gini', 'entropy'])
    max_features = trial.suggest_categorical('max_features',  ['auto','sqrt','log2'])
    min_samples_split= trial.suggest_int(' min_samples_split', 2,  30, step=1)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20, step=1)
    
    #splitter = trial.suggest_categorical('splitter', ['best', 'random'])

    ### modeling with suggested params
    etc_model = ExtraTreesClassifier(n_estimators = n_estimators,
                                 max_depth = max_depth,
                                 max_leaf_nodes = max_leaf_nodes,
                                 criterion = criterion,
                                 max_features = max_features,
                                 min_samples_split = min_samples_split,
                                 min_samples_leaf = min_samples_leaf,
                                 #splitter= splitter,
                                 random_state = 42)
    ### fit
    etc_model.fit(X_train1, y_train1) # train on train data
    score = cross_val_score(etc_model,X_train1, y_train1, scoring="accuracy")
    accuracy_mean = score.mean()

    return accuracy_mean
#     y_pred = etc_model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     return accuracy
    
    
study = optuna.create_study(direction='maximize') # maximize accuracy
study.optimize(objective, n_trials=800)

In [ ]:
print(etc_study.best_trial.params)  # Show the best value.
optimized_etc =ExtraTreesClassifier(**etc_study.best_params)

In [ ]:
# Evaluate Model on Training data
train_eval = evaluate_model_train(optimized_etc, X_train, y_train)
print("Confusion Matrix is:\n", train_eval['cm'])
print ('Accuracy : ', train_eval['Acc'])
print('Sensitivity : ', train_eval['sen'])
print('Specificity : ', train_eval['spec'])
print("Matthews Correlation Coefficient is: ", train_eval['mcc'])
print("Precision value is: ", train_eval['prec_train'])
print("Recall value is: ", train_eval['recall_train'])
print('The area under curve is:', train_eval['AUC'])
print("F1 score is: ", train_eval['f1_train'])

In [ ]:
# Evaluate Model on Testing data
dtc_eval = evaluate_model_test(optimized_etc, X_test, y_test)
# Print result
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('F1 Score:', dtc_eval['f1'])
print('Area Under Curve:', dtc_eval['auc'])
print('Sensitivity : ', dtc_eval['sen'])
print('Specificity : ', dtc_eval['spec'])
print('MCC Score : ', dtc_eval['mcc'])
print('Confusion Matrix:\n', dtc_eval['cm'])

# CatBoost

In [ ]:
def objective(trial):
   
    folds=10
    scores = []
    test_accs = []
   
    kf = KFold(n_splits=folds, shuffle=True, random_state=92)
    
    # kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=92)
    for i, (train_index, test_index) in enumerate(kf.split(data_only,labels)):
        X_train0, X_test1 = data_only[train_index], data_only[test_index]
        y_train0, y_test1 = labels[train_index], labels[test_index]
    
        X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0, y_train0, test_size=0.2, random_state=92, shuffle=True)
        
       
  
        cat_model = CatBoostClassifier(
        iterations=trial.suggest_int("iterations", 1, 100),
        learning_rate=trial.suggest_float("learning_rate", 1e-3, 1e-1, log=True),
        depth=trial.suggest_int("depth", 4, 16),
        l2_leaf_reg=trial.suggest_float("l2_leaf_reg", 1e-8, 100.0, log=True),
        bootstrap_type=trial.suggest_categorical("bootstrap_type", ["Bayesian"]),
        random_strength=trial.suggest_float("random_strength", 1e-8, 10.0, log=True),
        bagging_temperature=trial.suggest_float("bagging_temperature", 0.0, 10.0),
        od_type=trial.suggest_categorical("od_type", ["IncToDec", "Iter"]),
        od_wait=trial.suggest_int("od_wait", 10, 80),
        verbose=False
    )
        
        
        model2 = cat_model.fit(X_train1, y_train1)
        
        
        y_pred = model2.predict(X_test1)
        
        test_acc = metrics.accuracy_score(y_test1, y_pred)
        test_accs.append(test_acc)
    
        print('fold',i)
      
        # print(test_accs)
        i=i+1
    return np.mean(test_accs)
        


study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100, timeout=10000000000)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
print(study_cbc.best_trial)

In [ ]:
print(study_cbc.best_trial.params)  # Show the best value.

In [ ]:
import catboost
from catboost import CatBoostClassifier
optimized_cbt = catboost.CatBoostClassifier("best_params")

In [ ]:
# Evaluate Model on Training data
train_eval = evaluate_model_train(optimized_cbc, X_train, y_train)
print("Confusion Matrix is:\n", train_eval['cm'])
print ('Accuracy : ', train_eval['acc'])
print('Sensitivity : ', train_eval['sen'])
print('Specificity : ', train_eval['spec'])
print("Matthews Correlation Coefficient is: ", train_eval['mcc'])
print("Precision value is: ", train_eval['prec_train'])
print("Recall value is: ", train_eval['recall_train'])
print('The area under curve is:', train_eval['AUC'])
print("F1 score is: ", train_eval['f1_train'])

In [ ]:
# Evaluate Model on Testing data
dtc_eval = evaluate_model_test(optimized_cbc, X_test, y_test)
# Print result
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('F1 Score:', dtc_eval['f1'])
print('Area Under Curve:', dtc_eval['auc'])
print('Sensitivity : ', dtc_eval['sen'])
print('Specificity : ', dtc_eval['spec'])
print('MCC Score : ', dtc_eval['mcc'])
print('Confusion Matrix:\n', dtc_eval['cm'])

# XGB Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier as XGB
import xgboost

In [ ]:
def objective(trial):
    
    
    folds=10
    scores = []
    test_accs = []
   
    kf = KFold(n_splits=folds, shuffle=True, random_state=92)
    
    # kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=92)
    for i, (train_index, test_index) in enumerate(kf.split(data_only,labels)):
        
        
    
        X_train0, X_test1 = data_only[train_index], data_only[test_index]
        y_train0, y_test1 = labels[train_index], labels[test_index]
    
        X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0, y_train0, test_size=0.01, random_state=92, shuffle=True)
     
      
    params = {
           'max_depth': trial.suggest_int('max_depth', 1, 80),
           'learning_rate': trial.suggest_float('learning_rate', 0.01, 10.0),
           'n_estimators': trial.suggest_int('n_estimators', 10, 1000),
           'min_child_weight': trial.suggest_int('min_child_weight', 1, 40),
           'gamma': trial.suggest_float('gamma', 1e-8, 10.0),
           'subsample': trial.suggest_float('subsample', 0.01, 1.0),
           'colsample_bytree': trial.suggest_float('colsample_bytree', 0.01, 1.0),
           'reg_alpha': trial.suggest_float('reg_alpha', 1e-8, 1.0),
           'reg_lambda': trial.suggest_float('reg_lambda', 1e-8, 1.0),
           'eval_metric': 'mlogloss',
           'use_label_encoder': False
    }
    optuna_model = XGBClassifier(**params)
    optuna_model.fit(X_train1, y_train1)
    y_pred = optuna_model.predict(X_test1)
    accuracy = accuracy_score(y_test1, y_pred)
    return accuracy

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=700)



In [ ]:
print(XGB_study.best_trial)  # Show the best value.

In [ ]:
optimized_xgb = XGBClassifier(**XGB_study.best_params)

In [ ]:
# Evaluate Model on Training data
train_eval = evaluate_model_train(optimized_XGB, X_train, y_train)

print("Confusion Matrix is:\n", train_eval['cm'])
print ('Accuracy : ', train_eval['acc'])
print('Sensitivity : ', train_eval['sen'])
print('Specificity : ', train_eval['spec'])
print("Matthews Correlation Coefficient is: ", train_eval['mcc'])
print("Precision value is: ", train_eval['prec_train'])
print("Recall value is: ", train_eval['recall_train'])
print("F1 score is: ", train_eval['f1_train'])
print('The area under curve is:', train_eval['AUC'])

In [ ]:
# Evaluate Model on Testing data
dtc_eval = evaluate_model_test(optimized_XGB, X_test, y_test)

# Print result
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('F1 Score:', dtc_eval['f1'])
print('Area Under Curve:', dtc_eval['auc'])
print('Sensitivity : ', dtc_eval['sen'])
print('Specificity : ', dtc_eval['spec'])
print('MCC Score : ', dtc_eval['mcc'])
print('Confusion Matrix:\n', dtc_eval['cm'])

# LGBM

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.datasets import load_wine
import optuna

from optuna.samplers import TPESampler
import pickle
import shap

def objective(trial):
    
    folds=10
    scores = []
    test_accs = []
   
    kf = KFold(n_splits=folds, shuffle=True, random_state=92)
    
    # kf = StratifiedKFold(n_splits=folds, shuffle=True, random_state=92)
    for i, (train_index, test_index) in enumerate(kf.split(data_only,labels)):
        
        
    
        X_train0, X_test1 = data_only[train_index], data_only[test_index]
        y_train0, y_test1 = labels[train_index], labels[test_index]
    
        X_train1, X_validation, y_train1, y_validation = train_test_split(X_train0, y_train0, test_size=0.01, random_state=92, shuffle=True)
      
    
       
    
    param = {
        
        'num_leaves': trial.suggest_int('num_leaves', 2, 40, step=1, log=False), 
        'max_depth': trial.suggest_int('max_depth', 1, 80, step=1, log=False), 
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 1, log=True), 
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000, step=10), 
        #'objective': 'multiclass', 
        #'class_weight': trial.suggest_categorical('class_weight', ['balanced', None]),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 40, log=False), 
        'subsample': trial.suggest_uniform('subsample', 0.2, 1.0), 
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.2, 1.0),
        'reg_alpha': trial.suggest_uniform('reg_alpha', 0.0, 1.0),
        'reg_lambda': trial.suggest_uniform('reg_lambda', 0.0, 10.0),
        'random_state': 92
         }
    gbm = lgb.LGBMClassifier(**param)    
    gbm.fit(X_train1, y_train1)
    y_pred = gbm.predict(X_test1)
    accuracy = accuracy_score(y_test1, y_pred)
    return accuracy
    
sampler = TPESampler(seed=1)
study = optuna.create_study(study_name="lightgbm", direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=800)


In [ ]:
print(lgbm_study.best_trial)  # Show the best value.

In [ ]:
print('Best value:', study.best_value)

In [ ]:
optimized_gbm = lgb.LGBMClassifier(" best_params")

In [ ]:
%%time
# Evaluate Model on Training data
train_eval = evaluate_model_train(optimized_lgbm, X_train, y_train)
print("Confusion Matrix is: ", train_eval['cm'])
print ('Accuracy : ', train_eval['acc'])
print('Sensitivity : ', train_eval['sen'])
print('Specificity : ', train_eval['spec'])
print("Mean of Matthews Correlation Coefficient is: ", train_eval['mcc'])
print("The Precision value is: ", train_eval['prec_train'])
print("The Recall value is: ", train_eval['recall_train'])
print("The F1 score is: ", train_eval['f1_train'])
print('The area under curve is:', train_eval['AUC'])

In [ ]:
# Evaluate Model on Testing data
dtc_eval = evaluate_model_test(optimized_lgbm, X_test, y_test)
# Print result
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('F1 Score:', dtc_eval['f1'])
print('Area Under Curve:', dtc_eval['auc'])
print('Sensitivity : ', dtc_eval['sen'])
print('Specificity : ', dtc_eval['spec'])
print('MCC Score : ', dtc_eval['mcc'])
print('Confusion Matrix:\n', dtc_eval['cm'])

# MLP Classifier

In [ ]:
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=1)

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP= MLPClassifier(solver='adam',activation='logistic', alpha=1e-2, hidden_layer_sizes=(20), random_state=1)
MLP.fit(X_train, y_train)
scores = cross_val_score(MLP, X_train, y_train, cv=cv,  scoring='accuracy')
y_pred =MLP.predict(X_test_ind)
accuracy = accuracy_score(y_test_ind, y_pred)

print(scores.mean())
print(accuracy)


In [ ]:
%%time
# Evaluate Model on Training data
train_eval = evaluate_model_train(MLP, X_train, y_train)
print("Confusion Matrix is: ", train_eval['cm'])
print ('Accuracy : ', train_eval['acc'])
print('Sensitivity : ', train_eval['sen'])
print('Specificity : ', train_eval['spec'])
print("Mean of Matthews Correlation Coefficient is: ", train_eval['mcc'])
print("The Precision value is: ", train_eval['prec_train'])
print("The Recall value is: ", train_eval['recall_train'])
print("The F1 score is: ", train_eval['f1_train'])
print('The area under curve is:', train_eval['AUC'])

In [ ]:
# Evaluate Model on Testing data

dtc_eval = evaluate_model_test(MLP, X_test_ind, y_test_ind)
# Print result
print('Accuracy:', dtc_eval['acc'])
print('Precision:', dtc_eval['prec'])
print('Recall:', dtc_eval['rec'])
print('F1 Score:', dtc_eval['f1'])
print('Area Under Curve:', dtc_eval['auc'])
print('Sensitivity : ', dtc_eval['sen'])
print('Specificity : ', dtc_eval['spec'])
print('MCC Score : ', dtc_eval['mcc'])
print('Confusion Matrix:\n', dtc_eval['cm'])